In [13]:
import pandas as pd
import numpy as np
from pyspark import SparkContext
#Imports for Maps
import folium
from folium.plugins import MarkerCluster
#Helps to find Country names 
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [14]:
#Loading data from S3 bucket
lat_longs = sqlContext.read.option('delimiter', ' ').option('header','false').csv('s3://devicestatus1/lat_longs.txt')

In [15]:
#Displaying the data
lat_longs.head(5)

[Row(_c0='36.7', _c1='3.216666666666667', _c2='<http://dbpedia.org/resource/Algeria>'),
 Row(_c0='42.5', _c1='1.5166666666666666', _c2='<http://dbpedia.org/resource/Andorra>'),
 Row(_c0='12.516666666666667', _c1='-70.03333333333333', _c2='<http://dbpedia.org/resource/Aruba>'),
 Row(_c0='-8.833333333333334', _c1='13.333333333333334', _c2='<http://dbpedia.org/resource/Angola>'),
 Row(_c0='41.333333333333336', _c1='19.8', _c2='<http://dbpedia.org/resource/Albania>')]

In [16]:
#Changing the column names
lat_longs = lat_longs.withColumnRenamed("_c0","lat").withColumnRenamed("_c1","long").withColumnRenamed("_c2","name_of_page")

In [17]:
#Displaying the Data Frame with new column names
lat_longs.head(5)

[Row(lat='36.7', long='3.216666666666667', name_of_page='<http://dbpedia.org/resource/Algeria>'),
 Row(lat='42.5', long='1.5166666666666666', name_of_page='<http://dbpedia.org/resource/Andorra>'),
 Row(lat='12.516666666666667', long='-70.03333333333333', name_of_page='<http://dbpedia.org/resource/Aruba>'),
 Row(lat='-8.833333333333334', long='13.333333333333334', name_of_page='<http://dbpedia.org/resource/Angola>'),
 Row(lat='41.333333333333336', long='19.8', name_of_page='<http://dbpedia.org/resource/Albania>')]

In [18]:
#Creating a pandas data frame from spark data frame
pandas_df = lat_longs.select("*").toPandas()

In [19]:
#Showing pandas data frame
pandas_df.head()

,lat,long,name_of_page
0,36.7,3.216666666666667,<http://dbpedia.org/resource/Algeria>
1,42.5,1.5166666666666666,<http://dbpedia.org/resource/Andorra>
2,12.516666666666667,-70.03333333333333,<http://dbpedia.org/resource/Aruba>
3,-8.833333333333334,13.333333333333334,<http://dbpedia.org/resource/Angola>
4,41.333333333333336,19.8,<http://dbpedia.org/resource/Albania>


In [20]:
#Dropping rows with zero values
columns = ['lat', 'long']
pandas_df = pandas_df.replace(0, np.nan).dropna(axis=0, how='any', subset=columns).fillna(0)

In [21]:
#creating an empty map
world_map= folium.Map(tiles="cartodbpositron")
#Adding the Empty map to marker cluster
marker_cluster = MarkerCluster().add_to(world_map)

In [22]:
#Passing the Latitude and Longitude information to Folium lib by iterating and addting the results to marker cluster
lat_x = pandas_df['lat'].tolist()
lon_y = pandas_df['long'].tolist()
for i in range(len(lat_x)):
    radius=5
    folium.CircleMarker(location = [lat_x[i], lon_y[i]], radius=radius, fill =True).add_to(marker_cluster)

In [ ]:
#Printing the plotted latitude and logitude information on map
world_map